In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/fix-colab-gpu.sh && bash fix-colab-gpu.sh

## Prepare Java Kernel for Google Colab
Since Java is not natively supported by Colab, we need to run the following code to enable Java kernel on Colab.

1. Run the cell bellow (click it and press Shift+Enter),
2. (If training on CPU, skip this step) If you want to use the GPU with MXNet in DJL 0.10.0, we need CUDA 10.1 or CUDA 10.2.
Since Colab supports CUDA 10.1, we will have to follow some steps to setup the environment.
Refresh the page (press F5) and stay at Python runtime on GPU. Run the file fix-colab-gpu script.

And then ensure that you have switched to CUDA 10.1.
3. After that, switch runtime to Java and hardware to GPU.(Might require refreshing the page and switching runtime)

Now you can write Java code.

In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/colab_build.sh && bash colab_build.sh

# Attention Pooling: Nadaraya-Watson Kernel Regression
:label:`sec_nadaraya-watson`

Now you know the major components of attention mechanisms under the framework in :numref:`fig_qkv`.
To recapitulate,
the interactions between
queries (volitional cues) and keys (nonvolitional cues)
result in *attention pooling*.
The attention pooling selectively aggregates values (sensory inputs) to produce the output.
In this section,
we will describe attention pooling in greater detail
to give you a high-level view of
how attention mechanisms work in practice.
Specifically,
the Nadaraya-Watson kernel regression model
proposed in 1964
is a simple yet complete example
for demonstrating machine learning with attention mechanisms.


In [ ]:
%load ../utils/djl-imports
%load ../utils/plot-utils
%load ../utils/Functions.java
%load ../utils/Animator.java
%load ../utils/PlotUtils.java

In [ ]:
NDManager manager = NDManager.newBaseManager();

## Generating the Dataset

To keep things simple,
let us consider the following regression problem:
given a dataset of input-output pairs $\{(x_1, y_1), \ldots, (x_n, y_n)\}$,
how to learn $f$ to predict the output $\hat{y} = f(x)$ for any new input $x$?

Here we generate an artificial dataset according to the following nonlinear function with the noise term $\epsilon$:

$$y_i = 2\sin(x_i) + x_i^{0.8} + \epsilon,$$

where $\epsilon$ obeys a normal distribution with zero mean and standard deviation 0.5.
Both 50 training examples and 50 testing examples
are generated.
To better visualize the pattern of attention later, the training inputs are sorted.


In [ ]:
int nTrain = 50; // No. of training examples
NDArray xTrain = manager.randomUniform(0, 1, new Shape(nTrain)).mul(5).sort(); // Training inputs

In [ ]:
Function<NDArray, NDArray> f = x -> x.sin().mul(2).add(x.pow(0.8));
NDArray yTrain =
        f.apply(xTrain)
                .add(
                        manager.randomNormal(
                                0f,
                                0.5f,
                                new Shape(nTrain),
                                DataType.FLOAT32)); // Training outputs
NDArray xTest = manager.arange(0f, 5f, 0.1f); // Testing examples
NDArray yTruth = f.apply(xTest); // Ground-truth outputs for the testing examples
int nTest = (int) xTest.getShape().get(0); // No. of testing examples
System.out.println(nTest);

The following function plots all the training examples (represented by circles),
the ground-truth data generation function `f` without the noise term (labeled by "Truth"), and the learned prediction function (labeled by "Pred").


In [ ]:
public static Figure plot(
        NDArray yHat,
        String trace1Name,
        String trace2Name,
        String xLabel,
        String yLabel,
        int width,
        int height) {
    ScatterTrace trace =
            ScatterTrace.builder(
                            Functions.floatToDoubleArray(xTest.toFloatArray()),
                            Functions.floatToDoubleArray(yTruth.toFloatArray()))
                    .mode(ScatterTrace.Mode.LINE)
                    .name(trace1Name)
                    .build();

    ScatterTrace trace2 =
            ScatterTrace.builder(
                            Functions.floatToDoubleArray(xTest.toFloatArray()),
                            Functions.floatToDoubleArray(yHat.toFloatArray()))
                    .mode(ScatterTrace.Mode.LINE)
                    .name(trace2Name)
                    .build();

    ScatterTrace trace3 =
            ScatterTrace.builder(
                            Functions.floatToDoubleArray(xTrain.toFloatArray()),
                            Functions.floatToDoubleArray(yTrain.toFloatArray()))
                    .mode(ScatterTrace.Mode.MARKERS)
                    .marker(Marker.builder().symbol(Symbol.CIRCLE).size(15).opacity(.5).build())
                    .build();

    Layout layout =
            Layout.builder()
                    .height(height)
                    .width(width)
                    .showLegend(true)
                    .xAxis(Axis.builder().title(xLabel).domain(0, 5).build())
                    .yAxis(Axis.builder().title(yLabel).domain(-1, 5).build())
                    .build();

    return new Figure(layout, trace, trace2, trace3);
}

## Average Pooling

We begin with perhaps the world's "dumbest" estimator for this regression problem:
using average pooling to average over all the training outputs:

$$f(x) = \frac{1}{n}\sum_{i=1}^n y_i,$$
:eqlabel:`eq_avg-pooling`

which is plotted below. As we can see, this estimator is indeed not so smart.


In [ ]:
NDArray yHat = yTrain.mean().tile(nTest);
plot(yHat, "Truth", "Pred", "x", "y", 700, 500);

## Nonparametric Attention Pooling

Obviously,
average pooling omits the inputs $x_i$.
A better idea was proposed
by Nadaraya :cite:`Nadaraya.1964`
and Watson :cite:`Watson.1964`
to weigh the outputs $y_i$ according to their input locations:

$$f(x) = \sum_{i=1}^n \frac{K(x - x_i)}{\sum_{j=1}^n K(x - x_j)} y_i,$$
:eqlabel:`eq_nadaraya-watson`

where $K$ is a *kernel*.
The estimator in :eqref:`eq_nadaraya-watson`
is called *Nadaraya-Watson kernel regression*.
Here we will not dive into details of kernels.
Recall the framework of attention mechanisms in :numref:`fig_qkv`.
From the perspective of attention,
we can rewrite :eqref:`eq_nadaraya-watson`
in a more generalized form of *attention pooling*:

$$f(x) = \sum_{i=1}^n \alpha(x, x_i) y_i,$$
:eqlabel:`eq_attn-pooling`


where $x$ is the query and $(x_i, y_i)$ is the key-value pair.
Comparing :eqref:`eq_attn-pooling` and :eqref:`eq_avg-pooling`,
the attention pooling here
is a weighted average of values $y_i$.
The *attention weight* $\alpha(x, x_i)$
in :eqref:`eq_attn-pooling`
is assigned to the corresponding value $y_i$
based on the interaction
between the query $x$ and the key $x_i$
modeled by $\alpha$.
For any query, its attention weights over all the key-value pairs are a valid probability distribution: they are non-negative and sum up to one.

To gain intuitions of attention pooling,
just consider a *Gaussian kernel* defined as

$$
K(u) = \frac{1}{\sqrt{2\pi}} \exp(-\frac{u^2}{2}).
$$


Plugging the Gaussian kernel into
:eqref:`eq_attn-pooling` and
:eqref:`eq_nadaraya-watson` gives

$$\begin{aligned} f(x) &=\sum_{i=1}^n \alpha(x, x_i) y_i\\ &= \sum_{i=1}^n \frac{\exp\left(-\frac{1}{2}(x - x_i)^2\right)}{\sum_{j=1}^n \exp\left(-\frac{1}{2}(x - x_j)^2\right)} y_i \\&= \sum_{i=1}^n \mathrm{softmax}\left(-\frac{1}{2}(x - x_i)^2\right) y_i. \end{aligned}$$
:eqlabel:`eq_nadaraya-watson-gaussian`

In :eqref:`eq_nadaraya-watson-gaussian`,
a key $x_i$ that is closer to the given query $x$ will get
*more attention* via a *larger attention weight* assigned to the key's corresponding value $y_i$.

Notably, Nadaraya-Watson kernel regression is a nonparametric model;
thus :eqref:`eq_nadaraya-watson-gaussian`
is an example of *nonparametric attention pooling*.
In the following, we plot the prediction based on this
nonparametric attention model.
The predicted line is smooth and closer to the ground-truth than that produced by average pooling.


In [ ]:
// Shape of `xRepeat`: (`nTest`, `nTrain`), where each row contains the
// same testing inputs (i.e., same queries)
NDArray xRepeat = xTest.repeat(nTrain).reshape(new Shape(-1, nTrain));
// Note that `xTrain` contains the keys. Shape of `attention_weights`:
// (`nTest`, `nTrain`), where each row contains attention weights to be
// assigned among the values (`yTrain`) given each query
NDArray attentionWeights = xRepeat.sub(xTrain).pow(2).div(2).mul(-1).softmax(-1);
// Each element of `yHat` is weighted average of values, where weights are
// attention weights
yHat = attentionWeights.dot(yTrain);
plot(yHat, "Truth", "Pred", "x", "y", 700, 500);

Now let us take a look at the attention weights.
Here testing inputs are queries while training inputs are keys.
Since both inputs are sorted,
we can see that the closer the query-key pair is,
the higher attention weight is in the attention pooling.


In [ ]:
PlotUtils.showHeatmaps(
        attentionWeights.expandDims(0).expandDims(0),
        "Sorted training inputs",
        "Sorted testing inputs",
        new String[] {""},
        500,
        700);

## Parametric Attention Pooling

Nonparametric Nadaraya-Watson kernel regression
enjoys the *consistency* benefit:
given enough data this model converges to the optimal solution.
Nonetheless,
we can easily integrate learnable parameters into attention pooling.

As an example, slightly different from :eqref:`eq_nadaraya-watson-gaussian`,
in the following
the distance between the query $x$ and the key $x_i$
is multiplied a learnable parameter $w$:


$$\begin{aligned}f(x) &= \sum_{i=1}^n \alpha(x, x_i) y_i \\&= \sum_{i=1}^n \frac{\exp\left(-\frac{1}{2}((x - x_i)w)^2\right)}{\sum_{j=1}^n \exp\left(-\frac{1}{2}((x - x_i)w)^2\right)} y_i \\&= \sum_{i=1}^n \mathrm{softmax}\left(-\frac{1}{2}((x - x_i)w)^2\right) y_i.\end{aligned}$$
:eqlabel:`eq_nadaraya-watson-gaussian-para`

In the rest of the section,
we will train this model by learning the parameter of
the attention pooling in :eqref:`eq_nadaraya-watson-gaussian-para`.


### Batch Matrix Multiplication
:label:`subsec_batch_dot`

To more efficiently compute attention
for minibatches,
we can leverage batch matrix multiplication utilities
provided by deep learning frameworks.


Suppose that the first minibatch contains $n$ matrices $\mathbf{X}_1, \ldots, \mathbf{X}_n$ of shape $a\times b$, and the second minibatch contains $n$ matrices $\mathbf{Y}_1, \ldots, \mathbf{Y}_n$ of shape $b\times c$. Their batch matrix multiplication
results in
$n$ matrices $\mathbf{X}_1\mathbf{Y}_1, \ldots, \mathbf{X}_n\mathbf{Y}_n$ of shape $a\times c$. Therefore, given two tensors of shape ($n$, $a$, $b$) and ($n$, $b$, $c$), the shape of their batch matrix multiplication output is ($n$, $a$, $c$).


In [ ]:
NDArray X = manager.ones(new Shape(2, 1, 4));
NDArray Y = manager.ones(new Shape(2, 4, 6));

X.batchDot(Y).getShape()

In the context of attention mechanisms, we can use minibatch matrix multiplication to compute weighted averages of values in a minibatch.


In [ ]:
NDArray weights = manager.ones(new Shape(2, 10)).mul(0.1);
NDArray values = manager.arange(20f).reshape(new Shape(2, 10));
weights.expandDims(1).batchDot(values.expandDims(-1))

### Defining the Model

Using minibatch matrix multiplication,
below we define the parametric version
of Nadaraya-Watson kernel regression
based on the parametric attention pooling in
:eqref:`eq_nadaraya-watson-gaussian-para`.


In [ ]:
public class NWKernelRegression extends AbstractBlock {

    private Parameter w;
    public NDArray attentionWeights;

    public NWKernelRegression() {
        w = Parameter.builder().optShape(new Shape(1)).setName("w").optInitializer(new UniformInitializer()).build();
        addParameter(w);
    }

    @Override
    protected NDList forwardInternal(
            ParameterStore parameterStore,
            NDList inputs,
            boolean training,
            PairList<String, Object> params) {
        // Shape of the output `queries` and `attentionWeights`:
        // (no. of queries, no. of key-value pairs)
        NDArray queries = inputs.get(0);
        NDArray keys = inputs.get(1);
        NDArray values = inputs.get(2);
        queries =
                queries.repeat(keys.getShape().get(1))
                        .reshape(new Shape(-1, keys.getShape().get(1)));

        this.attentionWeights =
                queries.sub(keys).mul(this.w.getArray()).pow(2).div(2).mul(-1).softmax(-1);
        // Shape of `values`: (no. of queries, no. of key-value pairs)
        return new NDList(
                attentionWeights.expandDims(1).batchDot(values.expandDims(-1)).reshape(-1));
    }

    @Override
    public Shape[] getOutputShapes(Shape[] inputShapes) {
        throw new UnsupportedOperationException("Not implemented");
    }
}

### Training

In the following, we transform the training dataset
to keys and values to train the attention model.
In the parametric attention pooling,
any training input takes key-value pairs from all the training examples except for itself to predict its output.


In [ ]:
// Shape of `xTile`: (`nTrain`, `nTrain`), where each column contains the
// same training inputs
NDArray xTile = xTrain.tile(new long[] {nTrain, 1});
// Shape of `Y_tile`: (`nTrain`, `nTrain`), where each column contains the
// same training outputs
NDArray yTile = yTrain.tile(new long[] {nTrain, 1});
// Shape of `keys`: ('nTrain', 'nTrain' - 1)
NDArray keys =
        xTile.get(new NDIndex().addBooleanIndex(manager.eye(nTrain).mul(-1).add(1))).reshape(new Shape(nTrain, -1));
// Shape of `values`: ('nTrain', 'nTrain' - 1)
values = yTile.get(new NDIndex().addBooleanIndex(manager.eye(nTrain).mul(-1).add(1))).reshape(new Shape(nTrain, -1));

Using the squared loss and stochastic gradient descent,
we train the parametric attention model.


In [ ]:
NWKernelRegression net = new NWKernelRegression();
Loss loss = Loss.l2Loss();
Tracker lrt =
        Tracker.fixed(0.5f * nTrain); // Since we are using sgd, to be able to put the right
                                      // scale, we need to multiply by batchSize
Optimizer sgd = Optimizer.sgd().setLearningRateTracker(lrt).build();
DefaultTrainingConfig config =
        new DefaultTrainingConfig(loss)
                .optOptimizer(sgd) // Optimizer (loss function)
                .addTrainingListeners(TrainingListener.Defaults.logging()); // Logging
Model model = Model.newInstance("");
model.setBlock(net);

Trainer trainer = model.newTrainer(config);
Animator animator = new Animator();
ParameterStore ps = new ParameterStore(manager, false);


// Create trainer and animator
for (int epoch = 0; epoch < 5; epoch++) {
    try (GradientCollector gc = trainer.newGradientCollector()) {
        NDArray result = net.forward(ps, new NDList(xTrain, keys, values), true).get(0);
        NDArray l = trainer.getLoss().evaluate(new NDList(yTrain), new NDList(result));

        gc.backward(l);
        animator.add(epoch + 1, (float) l.getFloat(), "Loss");
        animator.show();
    }
    trainer.step();
}

After training the parametric attention model,
we can plot its prediction.
Trying to fit the training dataset with noise,
the predicted line is less smooth
than its nonparametric counterpart that was plotted earlier.


In [ ]:
// Shape of `keys`: (`nTest`, `nTrain`), where each column contains the same
// training inputs (i.e., same keys)
keys = xTrain.tile(new long[] {nTest, 1});

// Shape of `value`: (`nTest`, `nTrain`)
values = yTrain.tile(new long[] {nTest, 1});
yHat = net.forward(ps, new NDList(xTest, keys, values), true).get(0);
plot(yHat, "Truth", "Pred", "x", "y", 700, 500);

Comparing with nonparametric attention pooling,
the region with large attention weights becomes sharper
in the learnable and parametric setting.


In [ ]:
PlotUtils.showHeatmaps(
        net.attentionWeights.expandDims(0).expandDims(0),
        "Sorted training inputs",
        "Sorted testing inputs",
        new String[] {""},
        500,
        700);

## Summary

* Nadaraya-Watson kernel regression is an example of machine learning with attention mechanisms.
* The attention pooling of Nadaraya-Watson kernel regression is a weighted average of the training outputs. From the attention perspective, the attention weight is assigned to a value based on a function of a query and the key that is paired with the value.
* Attention pooling can be either nonparametric or parametric.


## Exercises

1. Increase the number of training examples. Can you learn  nonparametric Nadaraya-Watson kernel regression better?
1. What is the value of our learned $w$ in the parametric attention pooling experiment? Why does it make the weighted region sharper when visualizing the attention weights?
1. How can we add hyperparameters to nonparametric Nadaraya-Watson kernel regression to predict better?
1. Design another parametric attention pooling for the kernel regression of this section. Train this new model and visualize its attention weights.
